In [4]:
require 'hdf5';
require 'nn';

In [6]:
function build_model(dwin, nchar, nclass, hid1, hid2)
    -- Model with skip layer from Bengio, standards parameters
    -- should be:
    -- dwin = 5
    -- hid1 = 30
    -- hid2 = 100

    -- To store the whole model
    local dnnlm = nn.Sequential()

    -- Layer to embedd (and put the words along the window into one vector)
    local LT = nn.Sequential()
    local LT_ = nn.LookupTable(nchar,hid1)
    LT:add(LT_)
    LT:add(nn.View(-1, hid1*dwin))

    dnnlm:add(LT)

    local concat = nn.ConcatTable()

    local lin_tanh = nn.Sequential()
    lin_tanh:add(nn.Linear(hid1*dwin,hid2))
    lin_tanh:add(nn.Tanh())

    local id = nn.Identity()

    concat:add(lin_tanh)
    concat:add(id)

    dnnlm:add(concat)
    dnnlm:add(nn.JoinTable(2))
    dnnlm:add(nn.Linear(hid1*dwin + hid2, nclass))
    dnnlm:add(nn.LogSoftMax())

    -- Loss
    local criterion = nn.ClassNLLCriterion()

    return dnnlm, criterion
end


function train_model(train_input, train_output, dnnlm, criterion, dwin, nclass, eta, nEpochs, batchSize, val_space, val)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01

    -- To store the loss
    local av_L = 0

    -- Memory allocation
    local inputs_batch = torch.DoubleTensor(batchSize,dwin)
    local targets_batch = torch.DoubleTensor(batchSize)
    local outputs = torch.DoubleTensor(batchSize, nclass)
    local df_do = torch.DoubleTensor(batchSize, nclass)
    
    local true_kag = predict_kaggle(val_space)
    
    local tr = torch.Tensor(nEpochs)
    local vrmse = torch.Tensor(nEpochs)
    
    
    for i = 1, nEpochs do
        -- timing the epoch
        local timer = torch.Timer()

        av_L = 0
        
        -- max renorm of the lookup table
        dnnlm:get(1):get(1).weight:renorm(2,1,1)
        
        -- mini batch loop
        for t = 1, train_input:size(1), batchSize do
            -- Mini batch data
            local current_batch_size = math.min(batchSize,train_input:size(1)-t)
            inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
            targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
            
            -- reset gradients
            dnnlm:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
            outputs:narrow(1,1,current_batch_size):copy(dnnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

            -- Average loss computation
            local f = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
            av_L = av_L +f

            -- Backward pass
            df_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size)))
            dnnlm:backward(inputs_batch:narrow(1,1,current_batch_size), df_do:narrow(1,1,current_batch_size))
            dnnlm:updateParameters(eta)
            
        end
        Av = av_L/math.floor(train_input:size(1)/batchSize)
        tr[i] = Av
        print('Average Loss: '.. Av)
        
        tt = predict_space(dnnlm, val, dwin)
        tt_k = predict_kaggle(tt)
        
        rmse = torch.sqrt(torch.mean(torch.pow(tt_k:narrow(2,2,1)-true_kag:narrow(2,2,1),2)))
        vrmse[i] = rmse
        print('RMSE on valid: '.. rmse)
        
        print('Epoch '..i..': '..timer:time().real)
        
    end
    
    return tr, vrmse
end

In [7]:
function predict_space(nnlm, dat, Nwin)

    local dat_nospace = dat
    local s = dat:size(1)
    local it = 1
    local i = 1
    local nextpred = torch.Tensor(2)

    while it<s-(Nwin-1) do
        it = it + 1
        nextpred:copy(nnlm:forward(dat_nospace:narrow(1,i,Nwin)));
        m, argm = nextpred:max(1)

        if argm[1] == 2 then
            i = i + 1
        elseif argm[1] == 1 then 
            local dat_nospace_ = torch.LongTensor(dat_nospace:size(1)+1)
            dat_nospace_:narrow(1,1,i+(Nwin-1)):copy(dat_nospace:narrow(1,1,i+(Nwin-1)))
            dat_nospace_[i+Nwin] = 1
            dat_nospace_:narrow(1,i+(Nwin-1)+2,dat_nospace:size(1)-i-(Nwin-1)):copy(dat_nospace:narrow(1,i+(Nwin-1)+1,dat_nospace:size(1)-i-(Nwin-1)))
            dat_nospace = dat_nospace_
            i = i + 2
        end 
    end
    return dat_nospace
end

In [8]:
function predict_kaggle(dat_space)
    local num_sent = 0

    for i = 5,dat_space:size(1) do
        if dat_space[i] == 2 then
            num_sent = num_sent + 1
        end
    end

    local num_spaces = torch.DoubleTensor(num_sent,2)
    local row = 1
    local count_space = 0

    for i=5,dat_space:size(1) do
        if dat_space[i] == 2 then
            num_spaces[{row, 1}] = row
            num_spaces[{row, 2}] = count_space
            count_space = 0
            row = row + 1
        elseif dat_space[i] == 1 then
            count_space = count_space + 1
        end
    end

    return num_spaces
end

In [9]:
myFile = hdf5.open('../data_preprocessed/4-grams.hdf5','r')
data = myFile:all()
myFile:close()

N = 4

train_input = data['input_matrix_train']
train_output = data['output_matrix_train']
input_data_train = data['input_data_train']

input_data_valid = data['input_data_valid_nospace']:clone()
input_data_valid_space = data['input_data_valid']
input_data_test = data['input_data_test']:clone()

In [10]:
torch.manualSeed(1)
nnlm1, crit = build_model(N-1, 49, 2, 16, 80)
perp_train_3, rmse_valid_3 = train_model(train_input, train_output, nnlm1, crit, N-1, 2, 0.01, 15, 20,input_data_valid_space, input_data_valid)

Average Loss: 0.2999279016966	


RMSE on valid: 8.0990007022626	
Epoch 1: 37.549407958984	


Average Loss: 0.26833430403236	


RMSE on valid: 7.2910685526007	
Epoch 2: 37.056308031082	


Average Loss: 0.24193435098863	


...

Average Loss: 0.19310638144052	


RMSE on valid: 4.408529203773	
Epoch 13: 38.824916124344	


Average Loss: 0.19163403265493	


RMSE on valid: 4.3576394935891	
Epoch 14: 37.849529027939	


Average Loss: 0.19025300134216	


In [11]:
myFile = hdf5.open('../data_preprocessed/5-grams.hdf5','r')
data = myFile:all()
myFile:close()

N = 5

train_input = data['input_matrix_train']
train_output = data['output_matrix_train']
input_data_train = data['input_data_train']

input_data_valid = data['input_data_valid_nospace']:clone()
input_data_valid_space = data['input_data_valid']
input_data_test = data['input_data_test']:clone()

In [12]:
torch.manualSeed(1)
nnlm2, crit = build_model(N-1, 49, 2, 16, 80)
perp_train_4, rmse_valid_4 = train_model(train_input, train_output, nnlm2, crit, N-1, 2, 0.01, 15, 20, input_data_valid_space, input_data_valid)

Average Loss: 0.29443654451786	


RMSE on valid: 7.5064842229383	
Epoch 1: 36.957608938217	


Average Loss: 0.27378908120055	


RMSE on valid: 7.0633008969517	
Epoch 2: 38.712803840637	


Average Loss: 0.24403272683775	


...

RMSE on valid: 4.3477803087063	
Epoch 13: 41.250059127808	


Average Loss: 0.16932815903052	


RMSE on valid: 4.3262647707078	
Epoch 14: 38.601723909378	


Average Loss: 0.16800128690437	


RMSE on valid: 4.3135584413736	
Epoch 15: 39.222057104111	


In [13]:
myFile = hdf5.open('../data_preprocessed/6-grams.hdf5','r')
data = myFile:all()
myFile:close()

N = 6

train_input = data['input_matrix_train']
train_output = data['output_matrix_train']
input_data_train = data['input_data_train']

input_data_valid = data['input_data_valid_nospace']:clone()
input_data_valid_space = data['input_data_valid']
input_data_test = data['input_data_test']:clone()

In [14]:
torch.manualSeed(1)
nnlm3, crit = build_model(N-1, 49, 2, 16, 80)
perp_train_5, rmse_valid_5 = train_model(train_input, train_output, nnlm3, crit, N-1, 2, 0.01, 15, 20, input_data_valid_space, input_data_valid)

Average Loss: 0.295621389953	


RMSE on valid: 7.0153808779193	
Epoch 1: 38.690722942352	


Average Loss: 0.27468023492365	


RMSE on valid: 6.7282583985638	
Epoch 2: 38.674744129181	


Average Loss: 0.25618872109793	


...

RMSE on valid: 4.2452274483029	
Epoch 13: 38.151856184006	


Average Loss: 0.16307442759655	


RMSE on valid: 4.2755630181014	
Epoch 14: 42.683506965637	


Average Loss: 0.16100459655088	


RMSE on valid: 4.2954741070563	
Epoch 15: 38.342741966248	


In [15]:
myFile = hdf5.open('toplot.f5', 'w')
myFile:write('perp3', perp_train_3)
myFile:write('rmse3', rmse_valid_3)
myFile:write('perp4', perp_train_4)
myFile:write('rmse4', rmse_valid_4)
myFile:write('perp5', perp_train_5)
myFile:write('rmse5', rmse_valid_5)
myFile:close()

In [16]:
torch.manualSeed(1)
nnlm4, crit = build_model(N-1, 49, 2, 20, 80)
perp_train_6, rmse_valid_6 = train_model(train_input, train_output, nnlm4, crit, N-1, 2, 0.01, 15, 20, input_data_valid_space, input_data_valid)

Average Loss: 0.2949427931284	


RMSE on valid: 6.8002583020097	
Epoch 1: 38.400814056396	


Average Loss: 0.27373425207181	


RMSE on valid: 6.4982724558285	
Epoch 2: 43.661610126495	


Average Loss: 0.25465911687678	


...

Epoch 13: 42.313410043716	


Average Loss: 0.16392057574123	


RMSE on valid: 3.9816194460513	
Epoch 14: 43.537487983704	


Average Loss: 0.16237699344759	


RMSE on valid: 3.9523105246215	
Epoch 15: 39.288264036179	


In [17]:
myFile = hdf5.open('toplot2.f5', 'w')
myFile:write('perp6', perp_train_6)
myFile:write('rmse6', rmse_valid_6)
myFile:close()